# FireProx Batch Operations Guide

This notebook demonstrates Firestore batch operations in FireProx, enabling atomic multi-document writes without the complexity of transactions.

## What are Batches?

Batches allow you to **accumulate multiple write operations** and commit them **atomically** in a single request. Unlike transactions:

- ✅ **Write-only** - No read operations
- ✅ **No decorator required** - Simple, straightforward API
- ✅ **Atomic execution** - All operations succeed or all fail
- ✅ **Operation order guaranteed** - Writes execute in order added
- ✅ **More efficient** - Better for bulk writes
- ❌ **No automatic retry** - Unlike transactions
- ❌ **No read-modify-write** - Use transactions for that

## Batch Pattern

```python
# Create batch
batch = db.batch()

# Accumulate operations
doc1.field = 'value'
doc1.save(batch=batch)

doc2.delete(batch=batch)

# Commit atomically
batch.commit()  # or await batch.commit() for async
```

The demo is split into two sections:
1. Synchronous API examples
2. Asynchronous API examples

## Setup

Import modules and prepare for demonstrations.

In [1]:

from fire_prox import AsyncFireProx, FireProx
from fire_prox.testing import async_demo_client, demo_client

---

# Part 1: Synchronous Batch Operations

Examples using the synchronous FireProx API.

### Initialize Client and Create Sample Data

In [2]:
# Create sync client and collection
client = demo_client()
db = FireProx(client)
users = db.collection('batch_demo_users')

# Create initial users
alice = users.new()
alice.name = 'Alice'
alice.credits = 100
alice.status = 'active'
alice.save(doc_id='alice')

bob = users.new()
bob.name = 'Bob'
bob.credits = 50
bob.status = 'active'
bob.save(doc_id='bob')

charlie = users.new()
charlie.name = 'Charlie'
charlie.credits = 75
charlie.status = 'pending'
charlie.save(doc_id='charlie')

print("💰 Initial users:")
print(f"  Alice: {alice.credits} credits, status={alice.status}")
print(f"  Bob: {bob.credits} credits, status={bob.status}")
print(f"  Charlie: {charlie.credits} credits, status={charlie.status}")

💰 Initial users:
  Alice: 100 credits, status=active
  Bob: 50 credits, status=active
  Charlie: 75 credits, status=pending


## Feature 1: Basic Batch Updates

Update multiple documents atomically in a single batch.

In [3]:
# Create a batch
batch = db.batch()

# Update Alice
alice_ref = users.doc('alice')
alice_ref.fetch()
alice_ref.credits += 50
alice_ref.save(batch=batch)

# Update Bob
bob_ref = users.doc('bob')
bob_ref.fetch()
bob_ref.credits += 25
bob_ref.save(batch=batch)

# Commit all updates atomically
batch.commit()

print("📊 After batch update:")
alice_after = users.doc('alice')
alice_after.fetch(force=True)
print(f"  Alice: {alice_after.credits} credits")

bob_after = users.doc('bob')
bob_after.fetch(force=True)
print(f"  Bob: {bob_after.credits} credits")
print("\n✅ Both users updated atomically!")

📊 After batch update:
  Alice: 150 credits
  Bob: 75 credits

✅ Both users updated atomically!


## Feature 2: Mixed Operations (Set, Update, Delete)

Combine different operation types in a single batch.

In [4]:
# Create a new user to demonstrate mixed operations
dave = users.new()
dave.name = 'Dave'
dave.credits = 30
dave.status = 'inactive'
dave.save(doc_id='dave')

# Create batch with mixed operations
batch = db.batch()

# 1. Update Alice's status
alice_ref = users.doc('alice')
alice_ref.fetch()
alice_ref.status = 'premium'
alice_ref.save(batch=batch)

# 2. Activate Charlie
charlie_ref = users.doc('charlie')
charlie_ref.fetch()
charlie_ref.status = 'active'
charlie_ref.credits += 25
charlie_ref.save(batch=batch)

# 3. Delete inactive Dave
dave_ref = users.doc('dave')
dave_ref.delete(batch=batch)

# Commit all operations
batch.commit()

print("🔄 After mixed batch operations:")
alice_after = users.doc('alice')
alice_after.fetch(force=True)
print(f"  Alice: status={alice_after.status}")

charlie_after = users.doc('charlie')
charlie_after.fetch(force=True)
print(f"  Charlie: status={charlie_after.status}, credits={charlie_after.credits}")

print("  Dave: deleted ✅")
print("\n✅ Mixed operations completed atomically!")

🔄 After mixed batch operations:
  Alice: status=premium
  Charlie: status=active, credits=100
  Dave: deleted ✅

✅ Mixed operations completed atomically!


## Feature 3: Batch with Atomic Operations

Use atomic operations (ArrayUnion, ArrayRemove, Increment) within batches.

In [5]:
# Add tags and visit counters to users
alice_ref = users.doc('alice')
alice_ref.fetch()
alice_ref.tags = ['python', 'javascript']
alice_ref.visits = 10
alice_ref.save()

bob_ref = users.doc('bob')
bob_ref.fetch()
bob_ref.tags = ['python']
bob_ref.visits = 5
bob_ref.save()

print("Initial state:")
print(f"  Alice: tags={alice_ref.tags}, visits={alice_ref.visits}")
print(f"  Bob: tags={bob_ref.tags}, visits={bob_ref.visits}")

# Use batch with atomic operations
batch = db.batch()

# Add tags to Alice
alice_ref = users.doc('alice')
alice_ref.fetch()
alice_ref.array_union('tags', ['rust', 'go'])
alice_ref.increment('visits', 1)
alice_ref.save(batch=batch)

# Update Bob's tags
bob_ref = users.doc('bob')
bob_ref.fetch()
bob_ref.array_union('tags', ['typescript'])
bob_ref.increment('visits', 1)
bob_ref.save(batch=batch)

# Commit atomically
batch.commit()

print("\n🎯 After atomic operations batch:")
alice_after = users.doc('alice')
alice_after.fetch(force=True)
print(f"  Alice: tags={alice_after.tags}, visits={alice_after.visits}")

bob_after = users.doc('bob')
bob_after.fetch(force=True)
print(f"  Bob: tags={bob_after.tags}, visits={bob_after.visits}")
print("\n✅ Atomic operations in batch succeeded!")

Initial state:
  Alice: tags=['python', 'javascript'], visits=10
  Bob: tags=['python'], visits=5

🎯 After atomic operations batch:
  Alice: tags=['python', 'javascript', 'rust', 'go'], visits=11
  Bob: tags=['python', 'typescript'], visits=6

✅ Atomic operations in batch succeeded!


## Feature 4: Bulk Operations

Create and update many documents efficiently with batches.

In [6]:
# Create 20 documents
print("Creating 20 users...")
for i in range(20):
    user = users.new()
    user.name = f'User{i}'
    user.score = i * 10
    user.save(doc_id=f'user_{i}')

print("✅ 20 users created")

# Update all in a single batch
print("\nUpdating all 20 users in one batch...")
batch = db.batch()

for i in range(20):
    user = users.doc(f'user_{i}')
    user.fetch()
    user.score += 100  # Bonus points for everyone!
    user.save(batch=batch)

batch.commit()

# Verify a sample
print("\n📊 Sample of updated scores:")
for i in [0, 5, 10, 19]:
    user = users.doc(f'user_{i}')
    user.fetch(force=True)
    print(f"  User{i}: score={user.score}")

print("\n✅ All 20 users updated in single atomic batch!")
print("💡 Batches are perfect for bulk operations!")

Creating 20 users...
✅ 20 users created

Updating all 20 users in one batch...

📊 Sample of updated scores:
  User0: score=100
  User5: score=150
  User10: score=200
  User19: score=290

✅ All 20 users updated in single atomic batch!
💡 Batches are perfect for bulk operations!


## Feature 5: Creating Batches from Different Objects

You can create batches from `db`, `collection`, or `document` objects.

In [7]:
# Method 1: From db
batch1 = db.batch()
print("✅ Created batch from db")

# Method 2: From collection
batch2 = users.batch()
print("✅ Created batch from collection")

# Method 3: From document
alice_ref = users.doc('alice')
batch3 = alice_ref.batch()
print("✅ Created batch from document")

print("\n💡 All methods create identical batch objects!")
print("   Choose whichever is most convenient for your code.")

✅ Created batch from db
✅ Created batch from collection
✅ Created batch from document

💡 All methods create identical batch objects!
   Choose whichever is most convenient for your code.


## Feature 6: Real-World Pattern - Bulk User Activation

A practical example: activating multiple pending users at once.

In [8]:
# Create some pending users
pending_users = ['emma', 'frank', 'grace', 'henry']
for user_id in pending_users:
    user = users.new()
    user.name = user_id.capitalize()
    user.status = 'pending'
    user.credits = 0
    user.save(doc_id=user_id)

print("Created 4 pending users")

# Activate all at once with welcome bonus
print("\n🎉 Activating all pending users with welcome bonus...")
batch = db.batch()

for user_id in pending_users:
    user = users.doc(user_id)
    user.fetch()
    user.status = 'active'
    user.credits = 50  # Welcome bonus!
    user.save(batch=batch)

batch.commit()

print("\n✅ All users activated:")
for user_id in pending_users:
    user = users.doc(user_id)
    user.fetch(force=True)
    print(f"  {user.name}: status={user.status}, credits={user.credits}")

print("\n💡 Batches ensure all-or-nothing activation!")

Created 4 pending users

🎉 Activating all pending users with welcome bonus...

✅ All users activated:
  Emma: status=active, credits=50
  Frank: status=active, credits=50
  Grace: status=active, credits=50
  Henry: status=active, credits=50

💡 Batches ensure all-or-nothing activation!


---

# Part 2: Asynchronous Batch Operations

Examples using the asynchronous AsyncFireProx API with async/await.

### Initialize Async Client and Create Sample Data

In [9]:
# Create async client and collection
async_client = async_demo_client()
async_db = AsyncFireProx(async_client)
async_users = async_db.collection('batch_demo_users_async')

# Create initial users
alice = async_users.new()
alice.name = 'Alice'
alice.credits = 100
alice.status = 'active'
await alice.save(doc_id='alice')

bob = async_users.new()
bob.name = 'Bob'
bob.credits = 50
bob.status = 'active'
await bob.save(doc_id='bob')

print("💰 Initial async users:")
print(f"  Alice: {alice.credits} credits")
print(f"  Bob: {bob.credits} credits")

💰 Initial async users:
  Alice: 100 credits
  Bob: 50 credits


## Feature 1: Basic Async Batch Updates

Update multiple documents with async/await pattern.

In [10]:
# Create async batch
batch = async_db.batch()

# Update Alice
alice_ref = async_users.doc('alice')
await alice_ref.fetch()
alice_ref.credits += 50
await alice_ref.save(batch=batch)

# Update Bob
bob_ref = async_users.doc('bob')
await bob_ref.fetch()
bob_ref.credits += 25
await bob_ref.save(batch=batch)

# Commit with await
await batch.commit()

print("📊 After async batch update:")
alice_after = async_users.doc('alice')
await alice_after.fetch(force=True)
print(f"  Alice: {alice_after.credits} credits")

bob_after = async_users.doc('bob')
await bob_after.fetch(force=True)
print(f"  Bob: {bob_after.credits} credits")
print("\n✅ Async batch update succeeded!")

📊 After async batch update:
  Alice: 150 credits
  Bob: 75 credits

✅ Async batch update succeeded!


## Feature 2: Async Mixed Operations

Combine updates and deletes in async batch.

In [11]:
# Create user to delete
charlie = async_users.new()
charlie.name = 'Charlie'
charlie.status = 'inactive'
await charlie.save(doc_id='charlie')

# Mixed batch operations
batch = async_db.batch()

# Update Alice
alice_ref = async_users.doc('alice')
await alice_ref.fetch()
alice_ref.status = 'premium'
await alice_ref.save(batch=batch)

# Delete Charlie
charlie_ref = async_users.doc('charlie')
await charlie_ref.delete(batch=batch)

# Commit
await batch.commit()

print("🔄 After async mixed operations:")
alice_after = async_users.doc('alice')
await alice_after.fetch(force=True)
print(f"  Alice: status={alice_after.status}")
print("  Charlie: deleted ✅")
print("\n✅ Async mixed batch succeeded!")

🔄 After async mixed operations:
  Alice: status=premium
  Charlie: deleted ✅

✅ Async mixed batch succeeded!


## Feature 3: Async Batch with Atomic Operations

In [12]:
# Setup users with tags
alice_ref = async_users.doc('alice')
await alice_ref.fetch()
alice_ref.tags = ['python']
alice_ref.score = 100
await alice_ref.save()

bob_ref = async_users.doc('bob')
await bob_ref.fetch()
bob_ref.tags = ['javascript']
bob_ref.score = 50
await bob_ref.save()

print("Initial state:")
print(f"  Alice: tags={alice_ref.tags}, score={alice_ref.score}")
print(f"  Bob: tags={bob_ref.tags}, score={bob_ref.score}")

# Async batch with atomic operations
batch = async_db.batch()

# Update Alice
alice_ref = async_users.doc('alice')
await alice_ref.fetch()
alice_ref.array_union('tags', ['rust'])
alice_ref.increment('score', 50)
await alice_ref.save(batch=batch)

# Update Bob
bob_ref = async_users.doc('bob')
await bob_ref.fetch()
bob_ref.array_union('tags', ['typescript'])
bob_ref.increment('score', 25)
await bob_ref.save(batch=batch)

await batch.commit()

print("\n🎯 After async atomic operations:")
alice_after = async_users.doc('alice')
await alice_after.fetch(force=True)
print(f"  Alice: tags={alice_after.tags}, score={alice_after.score}")

bob_after = async_users.doc('bob')
await bob_after.fetch(force=True)
print(f"  Bob: tags={bob_after.tags}, score={bob_after.score}")
print("\n✅ Async atomic batch succeeded!")

Initial state:
  Alice: tags=['python'], score=100
  Bob: tags=['javascript'], score=50

🎯 After async atomic operations:
  Alice: tags=['python', 'rust'], score=150
  Bob: tags=['javascript', 'typescript'], score=75

✅ Async atomic batch succeeded!


## Feature 4: Async Bulk Operations

Efficiently handle large numbers of documents.

In [13]:
# Create 20 documents
print("Creating 20 async users...")
for i in range(20):
    user = async_users.new()
    user.name = f'AsyncUser{i}'
    user.points = i * 5
    await user.save(doc_id=f'async_user_{i}')

print("✅ 20 async users created")

# Update all in async batch
print("\nUpdating all 20 users in async batch...")
batch = async_db.batch()

for i in range(20):
    user = async_users.doc(f'async_user_{i}')
    await user.fetch()
    user.points += 50
    await user.save(batch=batch)

await batch.commit()

# Verify sample
print("\n📊 Sample of async updated points:")
for i in [0, 10, 19]:
    user = async_users.doc(f'async_user_{i}')
    await user.fetch(force=True)
    print(f"  AsyncUser{i}: points={user.points}")

print("\n✅ All 20 async users updated atomically!")

Creating 20 async users...
✅ 20 async users created

Updating all 20 users in async batch...

📊 Sample of async updated points:
  AsyncUser0: points=50
  AsyncUser10: points=100
  AsyncUser19: points=145

✅ All 20 async users updated atomically!


---

## Summary

This demo showcased all batch features:

### ✅ Batch Pattern

**Synchronous**:
```python
batch = db.batch()
doc.save(batch=batch)
doc2.delete(batch=batch)
batch.commit()
```

**Asynchronous**:
```python
batch = db.batch()
await doc.save(batch=batch)
await doc2.delete(batch=batch)
await batch.commit()
```

### ✅ Key Features

1. **Basic Operations** - Set, update, delete in batches
2. **Mixed Operations** - Combine different operation types
3. **Atomic Operations** - ArrayUnion, ArrayRemove, Increment
4. **Bulk Operations** - Efficiently handle many documents
5. **Convenient Creation** - From db, collection, or document
6. **Both Sync and Async** - Full support for both patterns

### 💡 Best Practices

- Use batches for **write-only** bulk operations
- Use batches when you need **atomic execution** without reads
- Batches can contain **up to 500 operations**
- All operations execute **atomically** (all-or-nothing)
- Operations execute in **order added**
- Cannot save **DETACHED** documents in batch (create first)
- For **read-modify-write**, use transactions instead

### 🚀 Real-World Use Cases

1. **Bulk User Activation** - Activate multiple pending users at once
2. **Batch Cleanup** - Delete inactive accounts or old data
3. **Bulk Status Updates** - Change status across many documents
4. **Mass Notifications** - Update read status for multiple notifications
5. **Batch Point Awards** - Award points to multiple users
6. **Archival Operations** - Archive or migrate multiple documents

### 📚 Learn More

- **Tests**: See `tests/test_integration_batches.py` for examples
- **Async Tests**: See `tests/test_integration_batches_async.py`
- **Transactions**: See `transactions.ipynb` for read-modify-write patterns
- **Native API**: [Firestore Batches Documentation](https://cloud.google.com/firestore/docs/manage-data/transactions#batched-writes)